# Deep Q-Learning

In this notebook you will teach a __pytorch__ neural network to do Q-learning.

In [ ]:
# !pip install -r ./requirement

In [ ]:
from collections import deque
import random
import numpy as np
import gym

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

### Let's play some old videogames
![img](https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/nerd.png)

This time we're gonna apply approximate q-learning to an OpenAI game called CartPole. It's not the hardest thing out there, but it's definitely way more complex than anything we tried before.


## Environment

In [ ]:
env = gym.make("CartPole-v0").env
env.reset()
n_actions = env.action_space.n
state_dim = env.observation_space.shape

# plt.imshow(env.render("rgb_array"))
# env.close()

# Approximate Q-learning: building the network

To train a neural network policy one must have a neural network policy. Let's build it.


Since we're working with a pre-extracted features (cart positions, angles and velocities), we don't need a complicated network yet. In fact, let's build something like this for starters:

![img](https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/yet_another_week/_resource/qlearning_scheme.png)

For your first run, please only use linear layers (nn.Linear) and activations. Stuff like batch normalization or dropout may ruin everything if used haphazardly. 

Also please avoid using nonlinearities like sigmoid & tanh: agent's observations are not normalized so sigmoids may become saturated from init.

Ideally you should start small with maybe 1-2 hidden layers with < 200 neurons and then increase network size if agent doesn't beat the target score.

In [ ]:
network = nn.Sequential(
    nn.Linear(env.observation_space.shape[0], 128),
    nn.ReLU(),
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Linear(128, env.action_space.n)
).to(device)

# network.add_module('layer1', < ... >)

# <YOUR CODE: stack layers!!!1 >

# hint: use state_dim[0] as input size

In [ ]:
def get_action(state, epsilon=0):
    """
    sample actions with epsilon-greedy policy
    recap: with p = epsilon pick random action, else pick action with highest Q(s,a)
    """
    state = torch.tensor(state[None], dtype=torch.float32)
    q_values = network(state).detach().numpy()[0]

    # YOUR CODE
    if np.random.random() < epsilon:
        action = np.random.randint(len(q_values))
    else:
        action = np.argmax(q_values)

    return int(action) # int( < epsilon-greedily selected action > )

In [ ]:
s = env.reset()
assert tuple(network(torch.tensor([s]*3, dtype=torch.float32)).size()) == (
    3, n_actions), "please make sure your model maps state s -> [Q(s,a0), ..., Q(s, a_last)]"
assert isinstance(list(network.modules(
))[-1], nn.Linear), "please make sure you predict q-values without nonlinearity (ignore if you know what you're doing)"
assert isinstance(get_action(
    s), int), "get_action(s) must return int, not %s. try int(action)" % (type(get_action(s)))

# test epsilon-greedy exploration
for eps in [0., 0.1, 0.5, 1.0]:
    state_frequencies = np.bincount(
        [get_action(s, epsilon=eps) for i in range(10000)], minlength=n_actions)
    best_action = state_frequencies.argmax()
    assert abs(state_frequencies[best_action] -
               10000 * (1 - eps + eps / n_actions)) < 200
    for other_action in range(n_actions):
        if other_action != best_action:
            assert abs(state_frequencies[other_action] -
                       10000 * (eps / n_actions)) < 200
    print('e=%.1f tests passed' % eps)

### Q-learning via gradient descent

We shall now train our agent's Q-function by minimizing the TD loss:
$$ L = { 1 \over N} \sum_i (Q_{\theta}(s,a) - [r(s,a) + \gamma \cdot max_{a'} Q_{-}(s', a')]) ^2 $$


Where
* $s, a, r, s'$ are current state, action, reward and next state respectively
* $\gamma$ is a discount factor defined two cells above.

The tricky part is with  $Q_{-}(s',a')$. From an engineering standpoint, it's the same as $Q_{\theta}$ - the output of your neural network policy. However, when doing gradient descent, __we won't propagate gradients through it__ to make training more stable (see lectures).

To do so, we shall use `x.detach()` function which basically says "consider this thing constant when doingbackprop".

In [ ]:
def to_one_hot(y_tensor, n_dims=None):
    """ helper: take an integer vector and convert it to 1-hot matrix. """
    y_tensor = y_tensor.type(torch.LongTensor).view(-1, 1)
    n_dims = n_dims if n_dims is not None else int(torch.max(y_tensor)) + 1
    y_one_hot = torch.zeros(
        y_tensor.size()[0], n_dims).scatter_(1, y_tensor, 1)
    return y_one_hot


def where(cond, x_1, x_2):
    """ helper: like np.where but in pytorch. """
    return (cond * x_1) + ((1-cond) * x_2)

In [ ]:
def compute_td_loss(
    states, 
    actions, 
    rewards, 
    next_states, 
    is_done, 
    gamma=0.99, 
    check_shapes=False
):
    """ Compute td loss using torch operations only. Use the formula above. """
    # shape: [batch_size, state_size]
    states      = torch.tensor(states, dtype=torch.float32).to(device)
    # shape: [batch_size]
    actions     = torch.tensor(actions, dtype=torch.int32).to(device)
    # shape: [batch_size]
    rewards     = torch.tensor(rewards, dtype=torch.float32).to(device)
    # shape: [batch_size, state_size]
    next_states = torch.tensor(next_states, dtype=torch.float32).to(device)
    # shape: [batch_size]
    is_done     = torch.tensor(is_done, dtype=torch.float32).to(device)

    # get q-values for all actions in current states
    predicted_qvalues = network(states)

    # select q-values for chosen actions
    predicted_qvalues_for_actions = torch.sum(
        predicted_qvalues * to_one_hot(actions, n_actions), 
        dim=1
    )

    # compute q-values for all actions in next states
    predicted_next_qvalues = network(next_states) # YOUR CODE

    # compute V*(next_states) using predicted next q-values
    next_state_values = predicted_next_qvalues.max(1)[0] # YOUR CODE
    assert next_state_values.dtype == torch.float32

    # compute "target q-values" for loss - it's what's inside square parentheses in the above formula.
    target_qvalues_for_actions =  rewards + gamma * next_state_values # YOUR CODE

    # at the last state we shall use simplified formula: Q(s,a) = r(s,a) since s' doesn't exist
    target_qvalues_for_actions = where(
        is_done, rewards, target_qvalues_for_actions)

    # mean squared error loss to minimize
    loss = torch.mean(
        (predicted_qvalues_for_actions - target_qvalues_for_actions.detach()) ** 2)

    if check_shapes:
        assert predicted_next_qvalues.data.dim(
        ) == 2, "make sure you predicted q-values for all actions in next state"
        assert next_state_values.data.dim(
        ) == 1, "make sure you computed V(s') as maximum over just the actions axis and not all axes"
        assert target_qvalues_for_actions.data.dim(
        ) == 1, "there's something wrong with target q-values, they must be a vector"

    return loss

In [ ]:
# sanity checks
s = env.reset()
a = env.action_space.sample()
next_s, r, done, _ = env.step(a)
loss = compute_td_loss([s], [a], [r], [next_s], [done], check_shapes=True)
loss.backward()

assert len(loss.size()) == 0, "you must return scalar loss - mean over batch"
assert np.any(next(network.parameters()).grad.detach().numpy() !=
              0), "loss must be differentiable w.r.t. network weights"

### Experience replay buffer

![img](https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/exp_replay.png)

In [ ]:
class ReplayBuffer(object):
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        state      = np.expand_dims(state, 0)
        next_state = np.expand_dims(next_state, 0)
            
        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        state, action, reward, next_state, done = zip(*random.sample(self.buffer, batch_size))
        return np.concatenate(state), action, reward, np.concatenate(next_state), done
    
    def __len__(self):
        return len(self.buffer)

---

In [ ]:
batch_size = 32

def generate_session(t_max=1000, epsilon=0, train=False):
    """play env with approximate q-learning agent and train it at the same time"""
    total_reward = 0
    s = env.reset()

    for t in range(t_max):
        a = get_action(s, epsilon=epsilon if train else -1)
        next_s, r, done, _ = env.step(a)

        if train:
            opt.zero_grad()
            replay_buffer.push(s, a, r, next_s, done)
            if len(replay_buffer) > batch_size:
                s_, a_, r_, next_s_, done_ = replay_buffer.sample(batch_size)
                compute_td_loss(s_, a_, r_, next_s_, done_).backward()

                opt.step()

        total_reward += r
        s = next_s
        if done:
            break

    return total_reward

In [ ]:
replay_buffer = ReplayBuffer(1000)
opt = torch.optim.Adam(network.parameters(), lr=1e-4)
epsilon = 0.5

In [ ]:
valid_mean_rewards = []
for i in range(100):
    session_rewards_train = [
        generate_session(epsilon=epsilon, train=True) 
        for _ in range(100)
    ]
    session_rewards_valid = [
        generate_session(epsilon=epsilon, train=False) 
        for _ in range(100)
    ]
    print(
        "epoch #{:02d}\tmean reward (train) = {:.3f}\tepsilon = {:.3f}\tmean reward (valid) = {:.3f}".format(
        i, np.mean(session_rewards_train), epsilon, np.mean(session_rewards_valid))
    )

    epsilon *= 0.95 # 0.99
    assert epsilon >= 1e-4, "Make sure epsilon is always nonzero during training"

    valid_mean_rewards.append(np.mean(session_rewards_valid))
    if len(valid_mean_rewards) > 5 and np.mean(valid_mean_rewards[-5:]) > 300:
        print("You Win!")
        break

### How to interpret results


Welcome to the f.. world of deep f...n reinforcement learning. Don't expect agent's reward to smoothly go up. Hope for it to go increase eventually. If it deems you worthy.

Seriously though,
* __mean reward__ is the average reward per game. For a correct implementation it may stay low for some 10 epochs, then start growing while oscilating insanely and converges by ~50-100 steps depending on the network architecture. 
* If it never reaches target score by the end of for loop, try increasing the number of hidden neurons or look at the epsilon.
* __epsilon__ - agent's willingness to explore. If you see that agent's already at < 0.01 epsilon before it's is at least 200, just reset it back to 0.1 - 0.5.

---

In [ ]:
# record sessions
import gym.wrappers
env = gym.wrappers.Monitor(
    gym.make("CartPole-v0"),
    directory="videos_dqn", 
    force=True)
sessions = [generate_session(epsilon=0, train=False) for _ in range(100)]
env.close()

In [ ]:
# show video
from IPython.display import HTML
import os

video_names = list(
    filter(lambda s: s.endswith(".mp4"), os.listdir("./videos_dqn/")))

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format("./videos/"+video_names[-1]))  # this may or may not be _last_ video. Try other indices